# Data Import, Merge, Wrangle

We will be using the real dataset for extra-EU trade percentages for a few different years to illutrate the real-world usage of data import, cleanse, merge and wrangle.


In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
df

""


### Concatenating datasets

Let's incrementally read in the EU trade dataset

In [4]:
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    df = pd.concat([df, chunk])

df.head(3)

,"partner,currency,stk_flow,sitc06,geo\time",2014,2012,2010
0,"EXT_EU,EUR,EXP,SITC0-4A,AT",61.9,65.6,67
1,"EXT_EU,EUR,EXP,SITC0-4A,BE",53.8,85.8,92.4
2,"EXT_EU,EUR,EXP,SITC0-4A,BG",57.0,46.2,54.1



### Transforming a column 

Let's preprocess this dataset to split out the 1st column into more manageable pieces.

In [8]:
df = pd.DataFrame()

# let's figure out how to split out the 1st column's values
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    data_rows = [row for row in chunk.ix[:,0].str.split(',')]
    data_cols = chunk.columns[0].split(',')
    print(data_rows[:2], data_cols)
    break; 
    df = pd.concat([df, chunk])

([['EXT_EU', 'EUR', 'EXP', 'SITC0-4A', 'AT'], ['EXT_EU', 'EUR', 'EXP', 'SITC0-4A', 'BE']], ['partner', 'currency', 'stk_flow', 'sitc06', 'geo\\time'])


Looks like we are getting the right value split.  However, the last column name split looks odd.

In [9]:
df = pd.DataFrame()
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    data_rows = [row for row in chunk.ix[:,0].str.split(',')]
    # create a column split list generator
    data_cols = [col.split('\\')[0] for col in chunk.columns[0].split(',')]
    print(data_rows[:2], data_cols)
    break; 
    df = pd.concat([df, chunk])

([['EXT_EU', 'EUR', 'EXP', 'SITC0-4A', 'AT'], ['EXT_EU', 'EUR', 'EXP', 'SITC0-4A', 'BE']], ['partner', 'currency', 'stk_flow', 'sitc06', 'geo'])


The last column name is now called "geo" -- simple and elegant.

Now, we need to merge the list of lists to create the cleaned dataframe:

In [10]:
df = pd.DataFrame()
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    data_rows = [row for row in chunk.ix[:,0].str.split(',')]
    # create a column split list generator
    data_cols = [col.split('\\')[0] for col in chunk.columns[0].split(',')]
    clean_df = pd.DataFrame(data_rows, columns=data_cols)
    
    # now we can concat by "column" which means axis=1
    new_df = pd.concat([clean_df, chunk], axis=1)
    print(new_df)
    break; 
    df = pd.concat([df, chunk])

   partner currency stk_flow    sitc06 geo  \
0   EXT_EU      EUR      EXP  SITC0-4A  AT   
1   EXT_EU      EUR      EXP  SITC0-4A  BE   
2   EXT_EU      EUR      EXP  SITC0-4A  BG   
3   EXT_EU      EUR      EXP  SITC0-4A  CY   
4   EXT_EU      EUR      EXP  SITC0-4A  CZ   
5   EXT_EU      EUR      EXP  SITC0-4A  DE   
6   EXT_EU      EUR      EXP  SITC0-4A  DK   
7   EXT_EU      EUR      EXP  SITC0-4A  EA   
8   EXT_EU      EUR      EXP  SITC0-4A  EE   
9   EXT_EU      EUR      EXP  SITC0-4A  EL   
10  EXT_EU      EUR      EXP  SITC0-4A  ES   
11  EXT_EU      EUR      EXP  SITC0-4A  EU   
12  EXT_EU      EUR      EXP  SITC0-4A  FI   
13  EXT_EU      EUR      EXP  SITC0-4A  FR   
14  EXT_EU      EUR      EXP  SITC0-4A  HR   
15  EXT_EU      EUR      EXP  SITC0-4A  HU   
16  EXT_EU      EUR      EXP  SITC0-4A  IE   
17  EXT_EU      EUR      EXP  SITC0-4A  IT   
18  EXT_EU      EUR      EXP  SITC0-4A  LT   
19  EXT_EU      EUR      EXP  SITC0-4A  LU   
20  EXT_EU      EUR      EXP  SITC

We notice that we still have our own column[0] around -- let's drop it to clean up the DF.

In [12]:
df = pd.DataFrame()
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    data_rows = [row for row in chunk.ix[:,0].str.split(',')]
    # create a column split list generator
    data_cols = [col.split('\\')[0] for col in chunk.columns[0].split(',')]
    clean_df = pd.DataFrame(data_rows, columns=data_cols)
    
    # now we can concat by "column" which means axis=1
    new_df = pd.concat([clean_df, 
                        chunk.drop(chunk.columns[0], axis=1)], axis=1)
    print(new_df)
    break; 
    df = pd.concat([df, chunk])

   partner currency stk_flow    sitc06 geo  2014   2012   2010 
0   EXT_EU      EUR      EXP  SITC0-4A  AT   61.9  65.6     67 
1   EXT_EU      EUR      EXP  SITC0-4A  BE   53.8  85.8   92.4 
2   EXT_EU      EUR      EXP  SITC0-4A  BG   57.0  46.2   54.1 
3   EXT_EU      EUR      EXP  SITC0-4A  CY   79.1  60.7   61.4 
4   EXT_EU      EUR      EXP  SITC0-4A  CZ   58.3  66.7   59.1 
5   EXT_EU      EUR      EXP  SITC0-4A  DE   62.5  61.5   65.9 
6   EXT_EU      EUR      EXP  SITC0-4A  DK   12.8    14   12.2 
7   EXT_EU      EUR      EXP  SITC0-4A  EA   60.7  65.4   64.1 
8   EXT_EU      EUR      EXP  SITC0-4A  EE   67.9  62.8   51.8 
9   EXT_EU      EUR      EXP  SITC0-4A  EL   60.3  58.4     59 
10  EXT_EU      EUR      EXP  SITC0-4A  ES   61.8  63.7   75.6 
11  EXT_EU      EUR      EXP  SITC0-4A  EU   50.1  53.5   53.2 
12  EXT_EU      EUR      EXP  SITC0-4A  FI   42.4  40.7   47.7 
13  EXT_EU      EUR      EXP  SITC0-4A  FR   63.8  62.3   58.4 
14  EXT_EU      EUR      EXP  SITC0-4A  

This looks a lot **cleaner**! 

We are finally ready to run this for all input data:

In [30]:
df = pd.DataFrame()
for chunk in pd.read_csv('data/ext_lt_invcur.tsv', sep='\t', chunksize=100):
    data_rows = [row for row in chunk.ix[:,0].str.split(',')]
    # create a column split list generator
    data_cols = [col.split('\\')[0] for col in chunk.columns[0].split(',')]
    clean_df = pd.DataFrame(data_rows, columns=data_cols)
    
    # now we can concat by "column" which means axis=1
    new_df = pd.concat([clean_df, 
                        chunk.drop(chunk.columns[0], axis=1)], axis=1)
    df = pd.concat([df, new_df])
    
df.head(3)

,partner,currency,stk_flow,sitc06,geo,2014,2012,2010
0,EXT_EU,EUR,EXP,SITC0-4A,AT,61.9,65.6,67
1,EXT_EU,EUR,EXP,SITC0-4A,BE,53.8,85.8,92.4
2,EXT_EU,EUR,EXP,SITC0-4A,BG,57.0,46.2,54.1


We have concat data both horizontally and vertically, and able to clean up the column name as well as transforming a messy column by splitting down its components.

In [35]:
df.shape

(1320, 8)

In [33]:
df.describe(include='all')

,partner,currency,stk_flow,sitc06,geo,2014,2012,2010
count,1320,1320,1320,1320,1320,1320.000000,1320,1320
unique,2,5,2,4,33,NaN,518,471
top,EXT_EU,OTH,IMP,SITC5-8,UK,NaN,100,100
freq,1200,264,660,330,40,NaN,238,248
mean,NaN,NaN,NaN,NaN,NaN,39.998712,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,39.025858,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.275000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,28.650000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,75.800000,NaN,NaN


In [38]:
df.loc[df['sitc06']=='TOTAL']

90    False
91     True
92    False
93    False
94    False
95    False
96    False
97    False
98    False
99    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
      ...  
9     False
10    False
11    False
21    False
22    False
23    False
33    False
34    False
35    False
45    False
46    False
47    False
57    False
58    False
59    False
69    False
70    False
71    False
81    False
82    False
83    False
93    False
94    False
95    False
5     False
6     False
7     False
17    False
18    False
19    False
Name: geo, dtype: bool